# [실습] OpenAI API와 Tool Calling을 이용한 뉴스 요약 봇 만들기
- OpenAI API를 통해 OpenAI의 기능을 호출하고 활용

## 환경 구성하기

In [1]:
# openai API 라이브러리 설치
!pip install openai tiktoken --upgrade


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 라이브러리 로드
import openai
import os

# openai api key 설정
os.environ['OPENAI_API_KEY']  = ""
client = openai.OpenAI()

# API키 검증하기
try : client.models.lis(); print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")
except: print(f"OPENAI_API_KEY가 유효하지 않습니다.")

OPENAI_API_KEY가 유효하지 않습니다.


### 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기
  - 네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드
  - 검색어가 query인수로 주어지면, 관련도순으로 검색된 뉴스기사 30개의 정보를 전달하는 함수를 요청
  - 네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 상승함

In [4]:
prompt = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 'query'인수로 주어지면, 관련도순으로 검색된 
뉴스 기사 30개의 정보를 아래 포맷의 문자열로 변환하는 함수 get_news(query)를 만들어 주세요.
---
제목 : (뉴스 제목)
URL : (뉴스 링크)
내용 : (뉴스 내용)
---


# 검색어
query  ='생성형 AI'

# 검색 결과를 return하는 함수
def get_news(query):
...

result = get_news(query)
print(result)

'''

In [5]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages=[
        {
            "role":'system',
            'content' :'당신은 파이썬 코딩의 전문가입니다. 설명 없이 코드만 출력해주세요.'
        },
        {
            'role' : 'user',
            'content' : prompt
        }
    ]
)

APIConnectionError: Connection error.

In [6]:
import requests

# 네이버 개발자 센터에서 발급받은 Client ID와 Client Secret을 입력합니다.
CLIENT_ID = 'your_client_id'  # 본인의 Client ID로 변경
CLIENT_SECRET = 'your_client_secret'  # 본인의 Client Secret으로 변경

# 검색어(query)를 입력받아 관련 뉴스 30개를 가져오는 함수
def get_news(query):
    url = "https://openapi.naver.com/v1/search/news.json"
    
    params = {
        'query': query,  # 검색어
        'display': 30,    # 검색 결과 수 (최대 30개)
        'sort': 'sim'     # 관련도순으로 정렬 (기본값은 'sim', 관련도순)
    }
    
    headers = {
        'X-Naver-Client-Id': CLIENT_ID,
        'X-Naver-Client-Secret': CLIENT_SECRET
    }
    
    # 네이버 API 호출
    response = requests.get(url, headers=headers, params=params)
    
    # 응답이 성공적이면
    if response.status_code == 200:
        news_items = response.json().get('items', [])
        
        # 뉴스 기사 정보를 포맷에 맞게 출력
        result = ""
        for item in news_items:
            title = item['title'].replace('<b>', '').replace('</b>', '')  # 제목에서 HTML 태그 제거
            link = item['link']
            description = item['description'].replace('<b>', '').replace('</b>', '')  # 내용에서 HTML 태그 제거
            
            result += f"제목 : {title}\nURL : {link}\n내용 : {description}\n---\n"
        
        return result
    else:
        return f"Error: {response.status_code}"

# 검색어 예시
query = '생성형 AI'
result = get_news(query)
print(result)

Error: 401


### OpenAI API를 이용하여 요약하기
   - 실제로는 각각의 URL에 접속해서 뉴스 링크를 가져오는 것이 일반적이지만, 간단하게 검색 결과만을 활용한는 코드를 만든다.
   - query에 대한 뉴스 검색 결과를 요약

In [ ]:
def news_bot(messages) :
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=messages,
        temperature=0.1,
        max_tokens=2048
    )
    
    return response.choices[0].message.content

- 챗봇에 전달할 메시지는 두 개로 나눠짐
- 위에서 얻은 get_news(query)외, 사용자가 제시할 명령

In [ ]:
query ='삼성 라이온즈'
result = get_news(query)

response = news_bot([{
    'role' :'system',
    'content':f"""
    뉴스 검색 결과가 주어집니다.
    {query}에 대한 뉴스를 요약하세요.
    ---
    """
        },
        {
            'role':'user',
            "content" : result
        }
])

In [ ]:
# 토큰 수 체크
import tiktoken

tokenizer = tiktoken.encoding_for_model('gpt-4o-mini')

print("문자 수:", len(result))
print("토큰 수:", len(tokenizer.encode(result)))
result


- 다양한 지시사항을 통해 형식과 스타일을 바꿀 수 있다.

In [ ]:
query ='삼성 라이온즈'
result = get_news(query)

response = news_bot([{
    'role' :'system',
    'content':f"""
    ---
    위 전체 내용을 종합하여 {query}의 동향과 미래에 대해 논쟁하는 두 사람의 대화 내용을 만들어줘
    한 명은 전문적인 말투, 한 명은 어리숙한 말투를 사용하고, 형식은 아래와 같아.
    ---
    A:(A의 대화)
    B:(B의 대화)
    """
        },
        {
            'role':'user',
            "content" : result
        }
])

### 스트리밍
- ChatGPT처럼 글자가 한 글자씩 순서대로 출력됨
- 이과 같은 작업을 스트리밍(Streaming)이라고 하며, 옵션을 추가하여 구현할 수 있다.

In [ ]:
def news_bot_stream(messages) :
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=messages,
        temperature=0.1,
        max_tokens=2048,
        stream=True # 스트리밍
    )
    
    return response # content가 아닌 전체를 return

In [ ]:
query ='LLM'
result = get_news(query)

response = news_bot_stream([
        {
            'role':'user',
            "content" : result
        },
        {
            'role' :'user',
            'content':f"""
            위 전체 내용을 종합하여 뉴스 리코팅을 해줘
            ---
            '제목' : 
            '본문' :
            """
        },
])

for chunk in result:
    print(chunk.choices[0].delta.content, end='')

- 스트리밍은 LLM  어플리케이션의 마지막 출력에서 사용하면 사용자 경험을 향상시킬 수 있다.

## Tool Calling : 외부 도구 사용하기
- 검색 기능을 LLM이 스스로 관리하도록 작성
- 사용자가 질문하면, LLM이 적절한 검색어를 구성하여 정보를 검색하고 답변
- Web_Searzh라는 이름의 툴을 만들고, 이를 get_news와 연결

In [7]:
from pydantic import BaseModel, Field

class Web_Search(BaseModel):
    """query를 이용하여 뉴스 검색"""
    query: str = Field(description= """
                                    검색 키워드 규칙 :
                                    1. 최대 2개 단어로 구성
                                    2. 불필요한 조사나 형용사 제외
                                    3. 핵심 명사만 포함
                                    
                                    예시 : 
                                    - (좋음) : "개봉영화", "영화
                                    - (나쁨) : "새로 개봉한 영화", "요즘 인기있는 영화
                                    """)
    tools = [openai.pydantic_function_tool(Web_Search)]
    tools

NameError: name 'Web_Search' is not defined

- Tool 정보가 포함된 gpt는 함수의 구조와 설명을 이용하여 어떤 함수를 써야 하는지를 판단
- 직접적으로 실행하는 것은 아니지만, 결과를 활용하면 해당 함수를 바로 사용할 수 있다.

In [ ]:
def news_bot_v2(messages, stream = False, model = 'gpt-4o-mini'):
    response = client.chat.completions.create(
        model= model,
        messages= messages,
        
        #### 사용할 툴 목록 전달
        tools = tools,
        tool_choice = "auto",
        # 'auto' : 자율적 툴 판단
        # 'none' : 툴 사용하지 않음
        # 'required' : 무조건 툴 사용
        ####
        
        temperature = 0.1,
        max_tokens = 1024,
        stream=stream
    )
    
    # Streaming 여부에 따라 출력 다르게 하기
    
    if stream : return response
    return response.choices[0].message

In [ ]:
# tool 사용 필요 없음

result = news_bot_v2([
    {
        "role" : "user",
        "content" : """안녕하세요! 오늘 날씨가 좋네요."""
    }
])

result

In [ ]:
# tool 사용 필요함

tool_call_result = news_bot_v2([
    {
        "role" : "user",
        "content" : """요즘 새로 개봉한 영화는 무엇이 있나요?"""
    }
])

tool_call_result

- Tool Call 메시지는 대신 tool_calls로 전달

In [ ]:
tool_call_result.tool_calls

- LLM이 스스로 판단하여, 실행할 함수의 정보를 tool_calls에 돌려준다.

#### tool_calls의 구성 요소
- id : tool call의 id로, 해당 id에 실행 결과를 연결할 수 있다.
- function : arguments와 name을 통해 실행할 툴의 이름과 매개변수를 전달

#### tool_calls의 결과를 임의로 포함하여 전달
- tool 타입의 메시지로 전달

In [ ]:
result = news_bot_v2([
    {
        "role" :"user",
        "content": """
                    요즘 새로 개봉한 영화는 무엇이 있나요?
                   """
    },
    tool_call_result, # tool_calls 메시지
    {
        "role" :"tool",
        "content": """
                    모아나2, 위키드, 무파사 : 라이온 킹, 수퍼 소닉3
                   """,
        "tool_call_id" :tool_call_result.tool_calls[0].id
    }
])

result

##### 실습

In [ ]:
name = tool_call_result.tool_calls[0].function.name
argument = tool_call_result.tool_calls[0].function.arguments

name, argument

- 우리의 목적은 'Web_Search', '{"query":"영화"}를 이용해 get_news(query="영화")를 실행하는 것

In [8]:
# 참고) 문자열을 Dict로 바꾸기
import json
example = '{"query":"영화"}'
example_dict = json.load(example)

# json으로 변환
print(example_dict)
print(type(example_dict))

In [ ]:
# 참고) 함수 이름 문자열을 함수로 바꾸기
available_functions = {'Web_Search': get_news}
available_functions['Web_Search']

In [ ]:
# 참고) Dictionary를 함수의 매개변수로 전달하기
# example_dict = {"query":"영화"}

get_news(**example_dict)

In [ ]:
# 3개 코드 모두 사용하면?
available_functions[name](**json.loads(argument))

###### 완성 코드

In [ ]:
available_functions = {'Web_Search':get_news}

name, argument = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls
search_result = available_functions[name](**json.loads(argument))

result = news_bot_v2([
    {
        "role" :"user",
        "content" : """
                    요즘 새로 개봉한 영화는 무엇이 있나요?
                    """
    },
    tool_call_result, # tool_calls 메시지
    {
        "role" :"tool",
        "content": search_result,
        "tool_call_id" :tool_call_result.tool_calls[0].id
    }
])

- 질문에 따라 적절한 쿼리를 생성하도록 코드를 수정
- 스트리밍 기능을 추가하고, 전체 과정을 함수로 구성

In [ ]:
def news_qa(prompt):
    available_functions = {"Web_Search" : get_news}
    print('Prompt : ', prompt)
    
    tool_call_result = news_bot_v2([
        {
            "role" :"user",
            "content" : prompt
        }
    ])
    # 프롬프트를 받아 Web_Search를 수행할지 결정
    # 단순 대화라면 tool call을 생성하지 않는다./
    
    print('---')
    print('News_Bot : Call', end='')
    
    if tool_call_result.tool_calls : # tool_calls가 존재하면:
        name, args = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls
        print(name, args)
        
        search_result = available_functions[name](**json.loads(args))
        print('---')
        print('News_Bot:', end='')
        
        response = news_bot_v2(
            [
                   {
                        "role" :"user",
                        "content" : prompt
                    },
                    tool_call_result, # tool_calls 메시지
                    {
                        "role" :"tool",
                        "content": search_result,
                        "tool_call_id" :tool_call_result.tool_calls[0].id
                    }
            ],
            stream=True # 최종 결과는 스트리밍
        )
        
        for chunk in response :
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)
    
prompt = """넷플릭스 신작 추천해줘."""

news_qa(prompt)

- 하나의 메시지에서 여러 개의 Tool Call을 수행해야 하기도 한다.

In [ ]:
# multiple tool call 필요

tool_call_result = news_bot_v2([
    {
        'role': 'user',
        'content' : """
                    넷플릭스 홧챠 디즈니플러스 웨이브 신작 추천해줘
                    """
    }
])

tool_call_result

- 반복문을 통해, Tool Call의 결과를 한꺼번에 전달
- 각각의 id와 결과를 mapping

In [ ]:
def news_qa_v2(prompt, model = 'gpt-4o-mini'):
    print('Prompt:', prompt)
    
    available_functions = {"Web_Search" :get_news}
    
    tool_call_result = news_bot_v2([
        {
            "role" : "user",
            "content" : prompt
        }
    ])
    
    print('---')
    print('News_Bot: Call ', end='')
    
    if tool_call_result.tool_calls: # tool_calls가 존재하면 :
        ### Tool Message 복수 입력
        
        tool_messages =[]
        
        # 여러 개의 tool_call에 대해, search_result 계산하여 리스트로 저장
        
        for tool_call in tool_call_result.tool_calls:
            name, args = tool_call.function.name, tool_call.function.arguments
            print(name, args)
            
            search_result = available_functions[name](**json.loads(args))
            
            print('---')
            print('News_Bot:', end='')
            
            tool_messages.append(
                {
                    "role" : "tool",
                    "content" : search_result,
                    "tool_call_id" : tool_call.id
                }
            )
            
            response = news_bot_v2([
                {
                 'role' : "user",
                 "content" : prompt
                },
                tool_call_result] + tool_messages,
                                   stream=True
                                   model=model
            )
            
            for chunk in response :
                print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)
        
prompt ="""넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘"""

news_qa_v2(prompt)

In [ ]:
news_qa_v2(prompt, model='gpt-4o')

### 한번에 입력되는 툴 출력이 너무 길어, 할루시네이션이 발생하는 모습
- Parellel Tool Calling을 비활성화하고, 하나씩 실행
  - 이 경우, Tool 호출 -> 메시지 전달 -> Tool 호출 -> 형태로 전달

In [ ]:
def news_bot_v3(messgaes, model = 'gpt-4o-mini'):
    # 사용할 툴 펑션의 목록과 설명을 리스트로 전달
    # LLM이 스스로 description, name, parameter의 값을 통해 판단
    
    response = client.chat.completions.create(
        model=model,
        messages=messgaes,
        
        tools = tools,
        tool_choice='auto',
        
        temperature=0,
        max_tokens=1024,

        # 툴 동시 실행 대신 번갈아 실행하기 (Tool -> ToolMsg -> Tool -> ToolMsg -> ...)        
        parallel_tool_calls=False,
    )
    
    return response.choices[0].message

In [ ]:

tool_call_result = news_bot_v3([
    {
        'role': 'user',
        'content' : """
                    넷플릭스 홧챠 디즈니플러스 웨이브 신작 추천해줘
                    """
    }
])

tool_call_result

In [ ]:
def news_qa_v3(prompt, model = 'gpt-4o-mini'):
    print('Prompt:', prompt)
    
    available_functions = {"Web_Search" :get_news}
    
    msgs = [
            {
            "role" : "user",
            "content" : prompt
        }
    ]
    
    tool_call_result = news_bot_v3(msgs)
    
    
    print('---')
    print('News_Bot: Call ', end='')
    
    while tool_call_result.tool_calls:
    # tool_calls --> tool msg --> tool_calls 형태이므로 while로 처리
        msgs.append(tool_call_result)
        name, args = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments
        print(name, args)
        
        search_result = available_functions[name](**json.load(args))
        print('---')
        print('News_Bot:', end='')
        
        msgs.append(
            {
                "role" : "tool",
                "content" : search_result,
                "tool_call_id": tool_call_result.tool_calls.id
            },
            tool_call_result = news_bot_v3(msgs, model = model)
        )
        
    print(tool_call_result.content)
    
   
        
prompt ="""넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘"""

news_qa_v3(prompt)